# Domain Specific IR System - Wiki Docs

This is a Wiki specific IR System to retrieve relevant Wiki docs based on an input User Query.
This code will will take a wiki file as input and perform the following:
##### 1. Tokenize the wiki document
##### 2. Apply stemming using porter stemmer
##### 3. Generate an Inverted Index
##### 4. Calculate TF-IDF Weights and Cosine Similarity
##### 5. Return the top 10 document IDs from the input User Query






Input : Wiki File (Path to be provided at line 17)


In [1]:
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
import nltk
from nltk import word_tokenize
import requests
import re
import math
import json

#Provide the path of the wiki file here
wikiPath = "C:/Users/dnv786/Desktop/Zebra/Personal/Mtech/Semester2/IR/Assignment/Sem2Assignment/wiki_00"

indexFilePath = 'InvertedIndex.json'

# METHOD DEFINITIONS

## 1. getUserQueryVector(userQuery)

This function takes a user query input and generates a vector from the query
This vector is then used to help in calculating Term Weights wrt Document terms in the Inverted Index


In [2]:
def getUserQueryVector(userQuery):
    """
    This method calculates Term frequency from the document
    """
    #First we tokenize the User Query just like how the wiki page was tokenized
    ps = PorterStemmer()
    queryTokens = nltk.word_tokenize(userQuery)
    stemmedQueryTokens = [ps.stem(eachWord) for eachWord in queryTokens]
#     queryTokens= [token.lower() for token in queryTokens if not re.match(r'^[_\W]+$', token)]
    cleanQueryTokens = []
    for eachQueryToken in stemmedQueryTokens:
        if not re.match(r'^[_\W]+$',eachQueryToken): #Checks if Punctuations are not present like [. , "''"] etc
            cleanQueryTokens.append(eachQueryToken.lower())
    
    queryTokenCount = {}
    for eachQueryToken in cleanQueryTokens:
        if eachQueryToken not in queryTokenCount:
            queryTokenCount[eachQueryToken] = 1
        else:
            queryTokenCount[eachQueryToken] += 1

    print("Query Tokens       : " + str(cleanQueryTokens))
    print("Query Token Count  : " + str(queryTokenCount))
    
    
    #Get the total number of documents in the wiki file. 
    #This we have already stored in the documentCount index in the InvertedIndex Data Structure
    totalDocumentsInInvertedIndex = InvertedIndex['documentCount']
    queryVector = {}
    
    
    
    
    ##THIS IS WHERE THE MAGIC HAPPENS
    ## Term Weight = (1+log(tf)) x (log(N/docFrequency))
     
    for token, count in queryTokenCount.items():
        if token in InvertedIndex["terms"]:
            
            # Term Frequency = (1+log(tf))
            termFrequency = 1 + math.log(count, 10)
            documentFrequency = InvertedIndex["terms"][token]['docs_count']
            
            # Inverse Document Frequency = log(N/docFrequency) , Where N - Total Documents in Inverted Index
            inverseDocumentFrequency = math.log(totalDocumentsInInvertedIndex/documentFrequency, 10)
            
            ## Term Weight = Term Frequency x Inverse Document Frequency
            termWeight = termFrequency * inverseDocumentFrequency
            
            ## Generate Query Vector
            if token not in queryVector:
                queryVector[token] = termWeight
            else: 
                queryVector[token] += termWeight

    return queryVector

## 2. retrieveTop10Docs(userQuery)

##### This is the Top level method that feeds in the user query

This will call the cosineScore and the getUserQueryVector methods to perform their corresponding operations.

This will finally retrieve the top 10 matching document IDs corresponding to the input User Query.



In [3]:
def retrieveTop10Docs(userQuery):
    userQueryVector = getUserQueryVector(userQuery)
    print("Query Vector : " + str(userQueryVector))
    scores = cosineScore(userQueryVector)
    top10Count = 0
    #Retrieve the first 10 documents from the document list that has the best scores matching the input sample query
    print('Top 10 Documents that match the sample input query, with corresponding scores : ')
    for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True):
        print(k, v)
        top10Count += 1
        if top10Count >= 10:
            break

## 3. cosineScore(userQueryVector)
This method takes the Vector generated from getUserQUeryVector method and then compares the cosine score between the user query and the document vector.

This method calculates the cosine score between vectors

In [4]:
def cosineScore(userQueryVector):
    scores = {}
    for token, qnorm in userQueryVector.items():
        for doc, data in InvertedIndex["terms"][token]['docs'].items():
            score = qnorm * data['tf_norm']
            if doc not in scores:
                scores[doc] = score
            else:
                scores[doc] += score
    return scores

## 4. termFrequency(doc, term)

This method calculates Term frequency from the document

In [5]:
def calculateTermFrequency(doc, term):
    if doc not in InvertedIndex["terms"][term]['docs']: return 1
    else:
        count = InvertedIndex["terms"][term]['docs'][doc]['count']
        termFrequency = 1 + math.log(count, 10)
    return termFrequency

# START OF MAIN PROGRAM

First Tokenize the documents from the input file

In [6]:
print("Reading Wiki File. This will take a few seconds...")
soup = BeautifulSoup(open(wikiPath, encoding="utf8"), "html.parser")
print("Reading Complete. Generated Beautiful soup object from Wiki File")

Reading Wiki File. This will take a few seconds...
Reading Complete. Generated Beautiful soup object from Wiki File


# Inverted Index Construction

Here we first define the structure for the Inverted Index which we need to create. 

The Beautiful Soup library is used to extract the data from any HTML/XML based files.

We also have to find each document in the Wiki File.

The Wiki File has a "doc" tag which We shall try to search in the Wiki File to identify each document.

And then we save the Document ID of the "doc" element from the wiki document

We will also be using Porter Stemmer operations to stem each token by creating a Porter Stemmer object

### Note :  Executing the following will take a while. Please be patient.

In [7]:
#Initializing the Inverted Index Structure
InvertedIndex = {'documentCount' : 0 , 'terms': {}}

#Create Porter Stemmer object for Stemming
ps = PorterStemmer()

print("Generating Tokens, Stemming and creating the Inverted Index.")
print("This will take a while. Please Wait till it says Complete...")

for eachDocument in soup.find_all('doc'):
    
    InvertedIndex['documentCount']+=1
    docID = int(eachDocument['id'])
    
    #Tokenize the Each document using  NLTK Tokenizer
    documentTokens = nltk.word_tokenize(eachDocument.get_text())
    
    #Apply Stemming operation using Porter Stemmer
    stemmedTokens = [ps.stem(eachWord) for eachWord in documentTokens]

    #Bring all tokens to lower case
    cleanTokens = []
    for eachToken in stemmedTokens:
        #Checks if Punctuations are not present like [. , "''"] etc
        if not re.match(r'^[_\W]+$',eachToken): 
            cleanTokens.append(eachToken.lower())
    
    for eachToken in cleanTokens:
        if eachToken not in InvertedIndex['terms']:
            #Initialize the Inverted Index structure for every New term
            InvertedIndex['terms'][eachToken] = {'docs' : {docID : {'count':1,'tf':0}},'total' : 1, 'docs_count':1}
        else:
            if docID not in InvertedIndex['terms'][eachToken]['docs']:
                InvertedIndex["terms"][eachToken]['docs'][docID] = {'count':1,'tf': 0}
                InvertedIndex["terms"][eachToken]['docs_count'] +=1
                InvertedIndex["terms"][eachToken]['total'] +=1
            else:
                InvertedIndex["terms"][eachToken]['docs'][docID]['count'] +=1
                InvertedIndex["terms"][eachToken]['total'] +=1
                
                

print("Tokenization, Stemming and Inverted Index Construction Complete")
# print(InvertedIndex)
# print(cleanTokens)


Generating Tokens, Stemming and creating the Inverted Index.
This will take a while. Please Wait till it says Complete...
Tokenization, Stemming and Inverted Index Construction Complete


### Calculate Term Frequency for each document in the wiki file represented by document ID

In [8]:
print("Calculating Term Frequency for each document")
for eachTerm in InvertedIndex["terms"]:
    for docID in InvertedIndex["terms"][eachTerm]['docs']:
        termFrequency = calculateTermFrequency(docID,eachTerm)
        InvertedIndex["terms"][eachTerm]['docs'][docID]['tf'] = termFrequency   
print("Complete!")

Calculating Term Frequency for each document
Complete!


### Normalize the Document Term Frequency using Cosine Coefficient

In [9]:
document_norm = {}
for eachTerm in InvertedIndex["terms"]:
    for docID in InvertedIndex["terms"][eachTerm]['docs']:
        termFrequency = InvertedIndex["terms"][eachTerm]['docs'][docID]['tf']
        if docID not in document_norm:
            document_norm[docID] = termFrequency**2
        else:
            document_norm[docID] += termFrequency**2

document_norm = {key: math.sqrt(value) for key, value in document_norm.items()}
print(document_norm)

{12: 61.875859443888146, 339: 52.505353112272786, 1023: 60.699152989882386, 25: 53.43784765878462, 39: 35.44412211157464, 290: 24.399519400571528, 303: 65.62818773801743, 305: 46.0532148714614, 307: 71.3852133094466, 308: 59.98122679110702, 309: 31.83072423580938, 316: 9.949820820695095, 324: 46.25194790156348, 330: 15.056443151316303, 332: 15.42739916403205, 334: 25.08520011949939, 336: 46.122755482729005, 340: 12.56747742179449, 344: 19.001173812856205, 358: 63.668811496658805, 359: 28.1154533068923, 569: 51.16606723308775, 572: 21.11996335643244, 573: 54.39452080499002, 580: 21.733382684246692, 586: 45.702928586802564, 593: 35.867354109104646, 594: 60.06713528318535, 595: 51.7463036265206, 597: 26.644473509724634, 599: 28.111489085655126, 600: 47.807783053496536, 612: 23.134068635871433, 615: 20.87935896341753, 620: 43.74191512815392, 621: 63.670273742499454, 624: 62.19346694605006, 627: 55.617313369608446, 628: 40.24756120917949, 633: 42.82645279526562, 634: 43.621228129506555, 639

### Add a new normalized term frequency field in the Inverted Index JSON Structure to store the Normalized Term Frequency

In [10]:
normalizedTermFrequencies = []
for eachTerm in InvertedIndex["terms"]:
    for docID in InvertedIndex["terms"][eachTerm]['docs']:
        InvertedIndex["terms"][eachTerm]['docs'][docID]['tf_norm'] = InvertedIndex["terms"][eachTerm]['docs'][docID]['tf'] / document_norm[docID]
        normalizedTermFrequencies.append(InvertedIndex["terms"][eachTerm]['docs'][docID]['tf_norm'] )

# print(str(normalizedTermFrequencies))
print("Complete!")

Complete!


## Finally Write the Inverted Index to a File


Write the final Inverted Index structure to a json file. 

'InvertedIndex.json'

This Inverted Index will be found in the same location where this python was run.

In [11]:
print("Writing Inverted Index... In Progress")
with open(indexFilePath,'w') as invertedIndexFile:
    invertedIndexFile.write(json.dumps(InvertedIndex))

print("Inverted Index File Write Completed")

Writing Inverted Index... In Progress
Inverted Index File Write Completed


# Input User Query Here

The userQuery will be the input to the IR system

The query will then be input to the top most method i.e. retrieveTop10Docs

#### This will then retrieve the IDs of the Top 10 documents of the sample Query

In [12]:
userQuery = "What is the meaning of Anarchism?"
retrieveTop10Docs(userQuery)

Query Tokens       : ['what', 'is', 'the', 'mean', 'of', 'anarch']
Query Token Count  : {'what': 1, 'is': 1, 'the': 1, 'mean': 1, 'of': 1, 'anarch': 1}
Query Vector : {'what': 0.34516395356044266, 'is': 0.03557615426119612, 'the': 0.03451818910486234, 'mean': 0.26814876926932557, 'of': 0.031359669860032684, 'anarch': 2.1712387562612694}
Top 10 Documents that match the sample input query, with corresponding scores : 
12 0.1306231786481284
1023 0.1093620818053134
339 0.08474952699982599
696 0.0661941066714508
1176 0.06263976031918828
1212 0.05136417468626282
1158 0.05088670516230048
643 0.04972585571201736
1160 0.04948656879034355
1309 0.0447406502511525
